In [1]:
import pandas as pd
import numpy as np
import torch 
import sys
sys.path.append("../../")
from dataset import *
from models import *
from trainer import *

In [2]:
claim_data = torch.Tensor(pd.read_pickle('../../data/claim_tensor.pkl'))
county_data = torch.Tensor(pd.read_pickle('../../data/county_tensor.pkl'))
hospitalizations_data = torch.Tensor(pd.read_pickle('../../data/hospitalizations.pkl'))
distance_matrix = torch.Tensor(pd.read_pickle('../../data/distance_mat.pkl'))
data_time = pd.read_pickle('../../data/date_range.pkl') #这个是list
claim_data.shape, county_data.shape, hospitalizations_data.shape, distance_matrix.shape,

In [3]:
dynamic_data = torch.cat((claim_data,torch.unsqueeze(hospitalizations_data, -1)), -1)
static_data = county_data
label = torch.unsqueeze(hospitalizations_data, -1)
dynamic_data = dynamic_data[:500]
static_data = static_data[:500]
label = label[:500]

dynamic_data.shape, static_data.shape, label.shape

In [4]:
preprocess = PreprocessData(dynamic_data = dynamic_data, static_data = static_data, label = label, input_window = 70, output_window = 30, type= 'temporal', mode = 0, method = 'mean')
trainset, valset, testset = preprocess.getMlDataSet()
train_temporal_loader = get_dataloader(trainset)
val_temporal_loader = get_dataloader(valset)
test_temporal_loader = get_dataloader(testset)

In [5]:
for x, y in train_temporal_loader:
    print(f'x.shape is {x.shape}, y.shape is {y.shape}')

In [6]:
xgb = XGBoost()
xgb_trainer = MlTrainer(xgb)
xgb_trainer.train(train_dataloader=train_temporal_loader)
xgb_trainer.evaluate(val_dataloader=val_temporal_loader)

In [8]:
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
import numpy as np

# 准备示例数据
X = np.random.rand(100, 5)  # 特征数据，形状 (100, 5)
y = np.random.rand(100, 3)  # 三个回归值，形状 (100, 3)

# 创建 XGBoost 回归模型
xgb_model = XGBRegressor()

# 使用 MultiOutputRegressor 将单输出模型包装成多输出模型
# multioutput_model = MultiOutputRegressor(xgb_model)

# 训练多输出模型
xgb_model.fit(X, y)

# 预测多个回归值
predictions = xgb_model.predict(X)
predictions.shape